# 预训练模型实战

**mask language model** pretrain case

## step1 导包

In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

## step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./wiki_cn_filtered")

In [3]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [4]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## step3 数据处理

In [5]:
model_path = r"D:\CodeLibrary\huggingface_model\hfl\chinese-macbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)

def process_func(examples):
    return tokenizer(examples["completion"], max_length=384, truncation=True)

`remove_columns=ds.column_names` 移除的是 ds的列名，即 'source' 和 'completion'

In [6]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

利用 `DataCollatorForLanguageModeling` 给数据做 `MASK` 编码

In [7]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15))

注意观察数据：
`dl` 中 `input_ids` 中的 `103` 的在 对应的 `labels` 中 **均不是**   `-100`

`-100` 表示的是不参与计算的 部分（可以是特殊符号也可以是一般符号）

In [9]:
# 利用内置函数 `next` 取一条数据查看格式
next(enumerate(dl))

(0,
 {'input_ids': tensor([[  101,   103,  2128,   769,  6858,  1920,  2110,  1300,  4289,  7667,
           8020, 13135,   112,  9064, 12095,  8731,  8626,  8181,  8736, 10553,
           8021,  3221,   103,  2429,   103,   754,  6205,  2128,   769,  6858,
           1920,  2110,  4638,  1300,  4289,  7667,   103,  7667,  7270,  3221,
            103,  3209,  1587,   511,  1325,  1380,  8258,  2399,   130,  3299,
           8113,  3189,  2458,  1993,  5040,  2456,  8024,  8138,  2399,   125,
           3299,   129,  3189,  3633,  2466,  2456,  2768,  2458,  7667,   103,
            855,   754,  6205,   103,   769,  6858,   103,  2110,  1069,  2412,
           3413,  1277,  7362,  6205,  4689,  6205,  2128,  2356,   103,  2123,
           6205,  6662,  8143,  1384,   511,  2456,  5029,  7481,  4916,   127,
            117,  8280,  2398,  5101,  8024,  2245,  1324,  7481,  4916,   125,
            103,  8195,  2398,  5101,  8024,  7667,  5966,  3152,   103,   103,
            117,  8567

In [10]:
tokenizer.mask_token, tokenizer.mask_token_id

('[MASK]', 103)

## step4 创建模型

In [13]:
model = AutoModelForMaskedLM.from_pretrained(model_path)

e:\software\anaconda3\envs\transformers\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at D:\CodeLibrary\huggingface_model\hfl\chinese-macbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model 

## step5 配置训练参数

In [14]:
args = TrainingArguments(
    output_dir="./masked_lm",
    per_device_train_batch_size=32,
    logging_steps=10,
    num_train_epochs=1
)

## step6 创建训练器

In [15]:
trainer = Trainer(
    model = model,
    args = args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15),
    train_dataset=tokenized_ds,
    tokenizer=tokenizer
)

## step7 训练

In [16]:
trainer.train()

Step,Training Loss
10,1.394700
20,1.409100
30,1.403400
40,1.363600
50,1.341300
60,1.314700
70,1.404700
80,1.325000
90,1.326600
100,1.389700


TrainOutput(global_step=313, training_loss=1.3283853660376308, metrics={'train_runtime': 6342.9217, 'train_samples_per_second': 1.577, 'train_steps_per_second': 0.049, 'total_flos': 1973819658240000.0, 'train_loss': 1.3283853660376308, 'epoch': 1.0})

## step8 模型推理

In [17]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [18]:
pipe("西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

[[{'score': 0.9972366094589233,
   'token': 1920,
   'token_str': '大',
   'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0010943046072497964,
   'token': 2110,
   'token_str': '学',
   'sequence': "[CLS] 西 安 交 通 学 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0001369447709294036,
   'token': 4906,
   'token_str': '科',
   'sequence': "[CLS] 西 安 交 通 科 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 7.870176341384649e-05,
   'token': 704,
   'token_str': '中',
   'sequence': "[CLS] 西 安 交 通 中 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 6.741079414496198e-05,
   'token': 7770,
   'token_str': '高',
   'sequence': "[CLS] 西 安 交 通 高 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}],
 [{'score': 0.99852377176284

In [19]:
pipe("今天天气[MASK]晴朗")

[{'score': 0.2012438029050827,
  'token': 2523,
  'token_str': '很',
  'sequence': '今 天 天 气 很 晴 朗'},
 {'score': 0.1865563541650772,
  'token': 1962,
  'token_str': '好',
  'sequence': '今 天 天 气 好 晴 朗'},
 {'score': 0.1216995120048523,
  'token': 6820,
  'token_str': '还',
  'sequence': '今 天 天 气 还 晴 朗'},
 {'score': 0.06360306590795517,
  'token': 679,
  'token_str': '不',
  'sequence': '今 天 天 气 不 晴 朗'},
 {'score': 0.048549965023994446,
  'token': 3221,
  'token_str': '是',
  'sequence': '今 天 天 气 是 晴 朗'}]